In [1]:
using Plots

In [2]:
Plots.gr()

Plots.GRBackend()

In [3]:
function evolve(Re,Im,Imold,V0,a,dx,dt,xmin,n)
    
    for i in 2:n
        x = xmin +(i-2)*dx
    
        HIm = calcV(x,V0,a)*Im[i] - 0.5(Im[i+1] -2Im[i] +Im[i-1])/dx^2
        
        # dtの整数倍の時刻で定義された実部
        Re[i] = Re[i] + HIm*dt
    end

    for i in 2:n
        x = xmin + (i-2)*dx
        
        # 実部よりdt/2だけ前の値
        Imold[i] = Im[i]
        HRe = calcV(x,V0,a)*Re[i] -0.5(Re[i+1]-2Re[i]+Re[i-1])/dx^2
        Im[i] = Im[i] - HRe*dt
    end
    
    return Re,Im,Imold
end

evolve (generic function with 1 method)

In [4]:
function initial_packet(Re,Im,x0,k0,width,xmin,n,dx,dt)
    
    # 初期状態としてのガウス型の波束
    
    delta2 = width^2
    A = (2π*delta2)^(-0.25)
    b = k0*dt/2
    
    for i in 1:n
        x = xmin + (i-1)*dx
        arg  = 0.25*(x-x0)^2/delta2
        arg2 = 0.25*(x-x0-0.5*k0*dt)^2/delta2

        Re[i] = A*cos(k0*(x-x0))*exp(-arg)
        Im[i] = A*sin(k0*(x-x0-0.5*b))*exp(-arg2)

    end
    
    return Re,Im
end

initial_packet (generic function with 1 method)

In [10]:
function calcV(x,V0,a)
   
    if abs(x) <= a
        V = 0
    else
        V = V0
    end
    
    return V
end

calcV (generic function with 1 method)

In [28]:
function main()
        
    x0    = 0            # 波束の初期位置
    width = 1              # x 空間での波束の幅
    k0    = 0              # 波束の群速度
    xmax  = 20
    xmin  = -xmax
    V0    = 10^4
    a     = 20              # 井戸型ポテンシャルの幅の半分
    dx    = 0.4            # 刻み幅
    n     = Int64((xmax-xmin)/dx)
    dt    = 0.1            # 時間間隔
    t = 0
    tmax  = 30
    
    plus= 10
    N = n+plus
    Re = zeros(Float64,N)
    Im = zeros(Float64,N)
    Imold = zeros(Float64,N)
    V = zeros(Float64,N)
    
    initial_packet(Re,Im,x0,k0,width,xmin,n,dx,dt)
    
    
    for i in 2:n
        x = xmin +(i-2)*dx
        V[i] = calcV(x,V0,a)
    end

#     plot(V)
#     plot(Re,label="Re(psi)")
#     plot!(Im,label="Im(psi)")
    
    unit = 10^2
    anim = @animate for i in 1:Int64(tmax/dt)
        
        evolve(Re,Im,Imold,V0,a,dx,dt,xmin,n)
        t += dt
        t = round(t*unit)/unit
        plot(V,label="potential")
        plot!(Re,label="Re(psi)")
        plot!(Im,label="Im(psi)")
        plot!(ylim=(-1,1),title="t=$t")
        
#         P = sum(Re.*Re + Im.*Imold)
#         P = P*dx
#         println(P)
        
    end
    
    w = width
    name = "k0=$k0,w =$w"
    
    gif(anim, "./$name.gif", fps = 15)
    

end
main()

┌ Info: Saved animation to 
│   fn = /Users/yoshiyuki/aqua/Applications to Physical Systems/aps_study/problem18_8/k0=0,w =1.gif
└ @ Plots /Users/yoshiyuki/.julia/packages/Plots/rmogG/src/animation.jl:90


Plots.AnimatedGif("/Users/yoshiyuki/aqua/Applications to Physical Systems/aps_study/problem18_8/k0=0,w =1.gif")